# Section A - Data Preparation 

The several analyzes developed require the application à priori of several data preparation methods. Fostering greater structure and interpretability of the project, the major methods are here considered, taking into account the different data sources used.


# 0 - Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from nltk.stem import WordNetLemmatizer
import re
from nltk.stem import RSLPStemmer
from nltk import word_tokenize
import string
from sklearn.decomposition import PCA
import nltk
import nlpnet
import swifter

%matplotlib inline
plt.style.use('ggplot')

# 1 - Traditional media 

The next section focuses in all the data preparation methods applied only on the traditional media dataset considered (newspaper articles published online).

## 1.1 - Data Import

### 1.1.1 - Data upload

In [2]:
original_dataset_media=pd.read_excel(r"C:\Users\franco joao\OneDrive - The Boston Consulting Group, Inc\From Egnyte\Personal\Programming\thesis_master\Data Lilia\JoaoF_Files\final_joao_franco_thesis_IJ\final_joao_franco_thesis_IJ\SDD_JI_Artigos.xlsx")

original_dataset_media.head()

,Unnamed: 0,title,lead,body,link
0,2007-01-03,Correio da Manhã,É capaz de ser uma brincadeira ritual do Bloco...,"Impresso do site do jornal Correio da manhã, e...",http://www.cmjornal.pt/opiniao/detalhe/lingua-...
1,2007-01-05,"Presidente diz que o país ""não pode descansar""","Na mensagem de ano novo, o Presidente da Repúb...","Na mensagem de ano novo, o Presidente da Repúb...",http://www.expresso.pt/africa/presidente-diz-q...
2,2007-01-07,Mercado accionista continua suportado,"Ricardo Valente, presidente da Personal Value ...","Impresso do site do jornal Correio da manhã, e...",http://www.cmjornal.pt/economia/detalhe/mercad...
3,2007-01-08,Falham investimentos portugueses,"""Moçambique não tem petróleo, logo não é tão i...","""Moçambique não tem petróleo, logo não é tão i...",http://www.expresso.pt/africa/falham-investime...
4,2007-01-12,Nótulas sobre o uso e desuso do guarda-chuva,Roupa para lavar,O Mundo divide-se em duas partes: os fãs do gu...,http://www.expresso.pt/blogues/blog_roupa_para...


### 1.1.2 - Legend

Legend of each column of the initial dataset used.

|Unnamed:0|title|lead|body|link|
|---|---|---|---|---|
|Date of publication of newspaper article|Title of the newspaper article|Resume (lead) of the newspaper articles|Main text (body) of the newspaper article|Web link to access the newspaper article|

## 1.2 - Data Treatment

In [3]:
original_dataset_media.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15898 entries, 0 to 15897
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  15898 non-null  datetime64[ns]
 1   title       15898 non-null  object        
 2   lead        15880 non-null  object        
 3   body        15898 non-null  object        
 4   link        15898 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 621.1+ KB


### 1.2.1 - Filling missing variables

In [4]:
original_dataset_media.lead.fillna(" ",inplace=True)
original_dataset_media.rename(columns={"Unnamed: 0":"Time"}, inplace=True)

In [5]:
original_dataset_media.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15898 entries, 0 to 15897
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Time    15898 non-null  datetime64[ns]
 1   title   15898 non-null  object        
 2   lead    15898 non-null  object        
 3   body    15898 non-null  object        
 4   link    15898 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 621.1+ KB


### 1.2.2 - Converting text to lowercase

In [6]:
#with the full string sets in lowercases we can simplified token search
for column in original_dataset_media.columns[[1,2,3]]:
    original_dataset_media[column]=[*map(str.lower,original_dataset_media[column])]

In [7]:
# Datasets of Sustainability (SDD) and Intergenerational Justice (JI)

data_media=original_dataset_media.copy()

columns=data_media.columns[[1,2,3]]
columns

Index(['title', 'lead', 'body'], dtype='object')

### 1.2.3 - Creating year, trimester and month related columns

In [8]:
def trimesters(dataset):

    trimester_JI=[]
    for month in dataset.month:
        if month<=3:
            trimester_JI.append(1)
        elif month >=10:
            trimester_JI.append(4)
        elif (month >=4 and month <=6):
            trimester_JI.append(2)
        else:
            trimester_JI.append(3)

    dataset["trimester"]=trimester_JI
    return dataset

In [9]:
data_media.loc[:,"year"]=pd.DatetimeIndex(data_media.Time).year
data_media.loc[:,"month"]=pd.DatetimeIndex(data_media.Time).month
data_media.loc[:,"trimester"]=trimesters(data_media)

### 1.2.4 - Removing non-Portuguese observations

It is important to ensure that **all the data is from articles published in Portugal**, removing the influence of other portuguese speaking countries (exemple: Brazil). 


We **considered that if the website is not ".pt", that it was not published in Portugal**.

In [10]:
data_media[~data_media.link.str.contains(".pt")].link

11497    http://publico.uol.com.br/lifestyle/artigo/est...
Name: link, dtype: object

With this, it is possible to conclude that only 1 observation can be considered as not published in Portugal.

In [11]:
data_media.loc[:,"PT_data"]=data_media.link.str.contains(".pt")

### 1.2.5 - Identifying JI or SDD related observations

In this section the different observations will be **classified as related with sustainability (SDD) or intergenerational justice (JI) , <u>depending on keywords frequency that were selected leveraging internal content knowledge<u>**.
  
**The presence of the topics in only one component** (body, title or lead) in the article **was considered enough to make the classification**, <u> minimizing false negative errors</u>. It is assumed that these terms are only used in cases strongly linked with the topics being studied, so even one occurrence is a strong indicator of the topic of the article.

The selected words to classify one observation as related with SDD or as related with JI were selected after an initial ad hoc analysis of the dataset.

In [12]:
for column in columns:
    data_media.loc[:,column + "SDD"]=data_media.loc[:,column].str.contains("sustentável|insustentável|sustentabilidade|insustentabilidade|sustentar")
    data_media.loc[:,column + "JI"]=data_media.loc[:,column].str.contains("interger|gerações vindouras|gerações futuras|geração futura|próxima geração|justiça entre gerações|geração vindoura|geração seguinte|gerações presentes|entre gerações|próximas gerações|gerações mais jovens| novas gerações| gerações distintas| próximas gerações")
    
data_media["SDD"]=data_media.titleSDD|data_media.leadSDD|data_media.bodySDD
data_media["JI"]=data_media.titleJI|data_media.leadJI|data_media.bodyJI

### 1.2.6 - JI or SDD related topic classification

When approaching SDD or JI, the writer can mention them in the **context of several topics (e.g.: environment, debt, health, among others)**. Therefore, it is important to understand within the observations classified as JI or SDD related, the topics that are related with. This is the focus of the second section.

Considering the same rational as for the JI or SDD observation classification method, the number of potential false negatives was considered minimized after a carefull manual consideration. In this case, lower bound assumed to consider one observation as related with debt, for example, was instead 2, considering an initial ad hoc analysis.


In order to facilitate the understanding of the topics that are being analysed, their will translated here:

- environment -> ambiente
- debt -> dívida
- education -> educação
- social security -> segurança social
- health -> saúde
- others -> outros

Although it is not a topic being study, there is other theme that is important to know its meaning in portuguese:

- politics -> política

In [13]:
environment=["ambiente","planeta","terra","água",
          "floresta","híbrido","incêndio","desfloresta","solo","erosão","reciclar","reutilizar"]
debt=["dívida","juros"]
education=["educação","escola","estudante","alunos","professores"]
health=["saúde","S.N.S"]

social_security=["pensão","segurança social", "S.S", "S.S.","SS", "pensões","reformado","reformada",
                  "pensionistas","contribuinte","discontos","rendimentos"]



others=["agricultura","agricultores","agrícola","polícia","TAP","RTP","RDP"]
politics=["assembleia da república","governo","ministro","partido"]

topics_name_w_politics=["environment","debt","education","health","social_security","others","politics"]

topics_name=["environment","debt","education","health","social_security","others"]

topics=[environment,debt,education,health,social_security,others,politics]

The function values_per_instances was developed to classify each article as related with one of the identified 6 topics, plus politics. 

In [14]:
#Key functions that will be used

def values_per_instances(instance,lists,lists_name,number):
    lists_counter=np.zeros(len(lists_name),dtype=int)
    result=[]
    for index_list,list_ in enumerate(lists):
        for word in list_:
            if instance.count(str(word))!=0:
                lists_counter[index_list]+=instance.count(str(word))
        if lists_counter[index_list]>=number:
            result.append(lists_name[index_list])
    return result

For an article to be classified as environment, health, etc. it is needed to **appear in each article at least two examples of words from that group**, being them unique or not. This reduces the levels of noise in the classification.

In [15]:
data_topics=data_media.body.swifter.apply(lambda x:values_per_instances(x,topics,topics_name_w_politics,2))

Pandas Apply:   0%|          | 0/15898 [00:00<?, ?it/s]

In [16]:
#cross check
len(data_topics)==len(data_media)

True

To convert the prior output into a dataframe with binary columns the update_database was created.

In [17]:
def update_database(content,lists_name):
        database=pd.DataFrame(index=np.arange(len(content)),columns=lists_name).fillna(0)
        for index_instance,instance in enumerate(content):
            for index_item,item in enumerate(instance):
                if item in lists_name:
                    database.loc[index_instance,item]+=1
        return database

In [18]:
data_topics_count=update_database(data_topics,topics_name_w_politics)

In [19]:
politics_info=data_topics_count.politics
#we remove the values from the politica column as politics is not considered as a topic
data_topics_count["total_topics"]=data_topics_count.sum(axis=1)-data_topics_count.politics
data_topics_count["year"]=data_media.year


In [20]:
#cross check
len(data_topics_count)==len(data_media)

True

In [21]:
#cross check
(data_topics_count.sum(axis=1)).value_counts().sum()==data_topics.value_counts().sum()


True

With data_topics_count DataFrame, we have the necessary information to compile the data_media DataFrame.

In [22]:
for topic in topics_name_w_politics:
    data_media.loc[:,topic]=data_topics_count.loc[:,topic]
    
data_media.loc[:,"total_topics"]=data_media.loc[:,topics_name_w_politics].sum(axis=1)-data_media.politics

### 1.2.7 - Tokenizer

Several analyses taken further will require the tokenization of the several newspaper articles, hence the **tokenization techniques required will be applied in this section** to optimize the code.


**Several limitations** are recognized, for example nltk is **trained not only from Portuguese from Portugal**, but also in texts from Portuguese from Brazil. Although the sentence construction can be different sometimes, **it is believed that the tokenization process would not be highly impacted**.


In [23]:
data_media.loc[:,"body_tokens"]=data_media.body.swifter.apply(word_tokenize)

Pandas Apply:   0%|          | 0/15898 [00:00<?, ?it/s]

### 1.2.8 - Political Party classification

A function was created to **classify each article as associated with a specific party**. **<u>This association is solely dependent on the number of mentions to a specific party</u>**. With this classification, we intend to know which are the parties that are more mentioned in the articles related to JI or SDD, and the respective topics. The fact that **some articles will be associated with more than 1 party is not a problem**, as there will cases where the same issue is discussed by different parties. In this case, **to reduce some noise, it was considered a minimum number of occurrences of each party name**. 2 occurences as in the topics classfication.

Although the <u>classification process is similar to the one used to classify observations as related to the different topics</u>, **the parties names (E.g.: "ps") can be to component of other words (E.g.: psychologist)**. Hence, a new method needed to be used to make the proper classification.

In [24]:

def exact_values_per_instances(instance,lists,lists_name,number):
    lists_counter=np.zeros(len(lists_name),dtype=int)
    result=[]
    for name in lists_name:
        counter=0
        #although it would be faster to search the tokens from in list with "lists", due to the case of PAN, explained below,
        #to ensure a complete analysis the search needs to be per token in the instance
        for word_index,word in enumerate(instance):
            if name!="PAN":
                if word in lists[name]:
                    counter+=1
            else:
                #this specific condition was inserted due to the particulary mention to the movie "peter pan" 
                #that does not refer at all to the political party pan
                if (word in lists[name]) & (instance[word_index-1]!="peter"):
                    counter+=1
        if counter>=number:
            result.append(name)
    return result
        
        
        
  

In [25]:
PS=["ps","socialista"]
PSD=["psd", "social democrata","sociais democratas"]
BE=["be"," bloco esquerda", "bloco de esquerda","bloquista"]
CDS=["cds","cds-pp ","cds-partido popular"]
PCP=["pcp", "cdu", " partido comunista ", "comunistas","partido comunista português"]
PEV=["pev","partido ecologista 'os verdes'"]
PAN=["pan", " partido das dessoas, dos animais e da natureza"]

parties=[PS,PSD,BE,CDS,PCP,PEV,PAN]
parties_name=["PS","PSD","BE","CDS","PCP","PEV","PAN"]

mixed_parties=dict(zip(parties_name,parties))

In [26]:
data_media_parties_aux=data_media.body_tokens.swifter.apply(lambda x:exact_values_per_instances(x,mixed_parties,parties_name,2))


Pandas Apply:   0%|          | 0/15898 [00:00<?, ?it/s]

In [27]:
data_media_parties_aux2=update_database(data_media_parties_aux,parties_name)

In [28]:
len(data_media_parties_aux2)==len(data_media)

True

In [29]:
for party in parties_name:
    data_media.loc[:,party]=data_media_parties_aux2.loc[:,party]
data_media.loc[:,"total_parties"]=data_media.loc[:,parties_name].sum(axis=1) 


### 1.2.9 - New feature: no_class_topic and no_class_party

Several analyses in the remaining documents require the identification of observations that are associated with no party or topic, so these 2 new features were created.

Several analyses in the remaining documents require the identification of observations that are associated with no party or topic, so these 2 new features were created.

In [30]:
no_class=[]
for index_,val in enumerate(data_media.index):
        if data_media.loc[:,topics_name].sum(axis=1).iloc[index_]==0:
            no_class.append(1)
        else:
            no_class.append(0)
            
data_media["no_class_topics"]=no_class

In [31]:
no_class=[]
for index_,val in enumerate(data_media.index):
        if data_media.loc[:,parties_name].sum(axis=1).iloc[index_]==0:
            no_class.append(1)
        else:
            no_class.append(0)
            
data_media["no_class_parties"]=no_class

### 1.2.10 - Filter for PT IJ or SDD related observations

Considering that all the analyses focus solely in observations related with:

1. IJ or SDD

2. With PT observations

The final dataset was filtered considering these 2 restrictions.

In [32]:
classified_data_media_SDDJI=data_media[(data_media.SDD|data_media.JI)&(data_media.PT_data)]

### 1.2.11 - Download to pickle document

In [33]:
classified_data_media_SDDJI.to_pickle("media_data_SDDJI.pkl")

data_media.to_pickle("media_data.pkl")

### 1.2.12 - Final Legend

|Time|title|lead|body|link|year|month|trimester|PT_data|titleSDD|titleJI|leadSDD|leadJI|bodySDD|bodyJI|SDD|JI|environment|debt|education|health|social_security|others|politics|total_topics|body_tokens|PS|PSD|BE|CDS|PCP|PEV|PAN|total_parties|no_class_topics|no_class_parties|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|Date of publication of newspaper article|Title of the newspaper article|Resume (lead) of the newspaper articles|Main text (body) of the newspaper article|Web link to access the newspaper article|Year of publication|Month of publication|Trimester of publication|Observation written in Portuguese from Portugal (Y/N)|Title classified as SDD related (Y/N)|Title classified as JI related (Y/N)|Lead classified as SDD related (Y/N)|Lead classified as JI related (Y/N)|Body classified as SDD related (Y/N)|Body classified as JI related (Y/N)|Title, or Lead, or Body classified as SDD related (Y/N)|Title, or Lead, or Body classified as JI related (Y/N)|Body classified as environment related (Y/N)|Body classified as debt related (Y/N)|Body classified as education related (Y/N)|Body classified as health related (Y/N)|Body classified as social security related (Y/N)|Body classified as related other topics (Y/N)|Body classified as politics related (Y/N)|Number of topics classification in the body text|All identified tokens in the body|Body classified as PS related (Y/N)|Body classified as PSD related (Y/N)|Body classified as BE related (Y/N)|Body classified as CDS related (Y/N)|Body classified as PCP related (Y/N)|Body classified as PEV related (Y/N)|Body classified as PAN related (Y/N)|Number of parties classifications in the body text|Body classified as not related with any topic (Y/N, 1 as not related)|Body classified as not related with any party (Y/N, 1 as not related)|

# 2 - Parliament

The next section focuses in all the data preparation methods applied only in the parliament dataset considered.

## 2.1 - Data import 

### 2.1.1 - Data upload

In [34]:
original_parliament_data=pd.read_pickle('classification_no_ins_1510.pkl')

### 2.1.2 - Legend

|person_id|gender|party|date|sustentabilidade|JI|ambiente|sáude|dívida|educação|SS|segurança|empresas|agricultura|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|Politicians id|Politician's gender (M/F)|Politician's political party|Speech date|Speech classified as SDD related (Y/N)|Speech classified as JI related (Y/N)|Speech classified as environment related (Y/N)|Speech classified as health related (Y/N)|Speech classified as debt related (Y/N)|Speech classified as education related (Y/N)|Speech classified as social security related (Y/N)|Speech classified as security (one of the topic "others") related (Y/N)|Speech classified as business (one of the topic "others") related (Y/N)|Speech classified as agriculture (one of the topic "others") related (Y/N)|

## 2.2 - Data Treatment

In [35]:
original_parliament_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112947 entries, 4177 to 117123
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   person__id        112947 non-null  object        
 1   gender            112947 non-null  object        
 2   party             112947 non-null  object        
 3   date              112947 non-null  datetime64[ns]
 4   sustentabilidade  112947 non-null  int32         
 5   JI                112947 non-null  int32         
 6   ambiente          112947 non-null  int32         
 7   saúde             112947 non-null  int32         
 8   dívida            112947 non-null  int32         
 9   educação          112947 non-null  int32         
 10  SS                112947 non-null  int32         
 11  segurança         112947 non-null  int32         
 12  empresas          112947 non-null  int32         
 13  agricultura       112947 non-null  int32         
dtypes

### 2.2.1 - Filling missing values for politician-party association

Although previously no null observations where identified, after a serious of analyses several observations it was observed that **some observations were filled with blank**. Although **the value does not confer any extract information it is not recognized as a null observation**.

In [36]:
for column in original_parliament_data.columns:
    print('' in original_parliament_data.loc[:,column].value_counts().index,column)

False person__id
False gender
True party
False date
False sustentabilidade
False JI
False ambiente
False saúde
False dívida
False educação
False SS
False segurança
False empresas
False agricultura


Afterwards, a party identification process was developed to fill the observations that have no party, as much as possible. 

The same person is sometimes associated with one specific party but other times it is associated with no party. So, **it was assumed that the same person always belonged to the party**, being possible to fill some blank observations.

In [37]:
ids_withno_party=original_parliament_data[original_parliament_data.party==''].person__id
ids_withno_party

4700      UGVyc29uTm9kZTo0Mzc=
4701      UGVyc29uTm9kZTo0Mzc=
4702      UGVyc29uTm9kZTo0Mzc=
4703      UGVyc29uTm9kZTo0Mzc=
4704      UGVyc29uTm9kZTo0Mzc=
                  ...         
116944    UGVyc29uTm9kZTozOTM=
116945    UGVyc29uTm9kZTozOTM=
116946    UGVyc29uTm9kZTozOTM=
117103    UGVyc29uTm9kZTozOTU=
117104    UGVyc29uTm9kZTozOTU=
Name: person__id, Length: 5631, dtype: object

For the observations with no politicia-party association, **the most recent one was considered was considered the most reliable one**. 

In [38]:
prior=original_parliament_data.party.value_counts()
prior.loc[""]

5631

In [39]:
for i,value in enumerate(ids_withno_party.index):
    subset_set_parliament=original_parliament_data.loc[original_parliament_data.person__id==ids_withno_party.loc[value]]
    list_parties=list(subset_set_parliament.party.drop_duplicates().values)
    if "" in list_parties: 
        list_parties.remove('')
        if len(list_parties)==1:
            aux_value=list_parties[0]
            original_parliament_data.update(original_parliament_data.loc[original_parliament_data.person__id==ids_withno_party.iloc[i]].party.replace("",aux_value))
        elif len(list_parties)>1:
            aux1_date=subset_set_parliament.loc[subset_set_parliament.index==value].date.loc[value]
            try:
                # the try method needs to be included as the blank might appear before any mention to this politician in specific
                aux_value=subset_set_parliament[(subset_set_parliament.date<=aux1_date)
                                                &(subset_set_parliament.party!="")].sort_values(by="date").iloc[-1].party
            except IndexError:
                pass
            original_parliament_data.loc[(original_parliament_data.person__id==ids_withno_party.iloc[i])
                                        &(original_parliament_data.date<=aux1_date)].party.replace("",aux_value,inplace=True)

C:\Users\franco joao\Anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
C:\Users\franco joao\Anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
C:\Users\franco joao\Anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace

In [40]:
# cross check to see that the replace worked
prior.loc[""],original_parliament_data.party.value_counts().loc[""]

(5631, 3041)

In [41]:
aux_CC=original_parliament_data.loc[original_parliament_data.party==""].person__id.value_counts().index
aux_CC

Index(['UGVyc29uTm9kZTozMzM2', 'UGVyc29uTm9kZTozNDU3', 'UGVyc29uTm9kZTozMzAw',
       'UGVyc29uTm9kZTozMzQz', 'UGVyc29uTm9kZTozMjQ0', 'UGVyc29uTm9kZTozMzQy',
       'UGVyc29uTm9kZTozMTcz', 'UGVyc29uTm9kZTozMzMx', 'UGVyc29uTm9kZTozMjIz',
       'UGVyc29uTm9kZTozMTk0',
       ...
       'UGVyc29uTm9kZTozNDU1', 'UGVyc29uTm9kZTozMjMx', 'UGVyc29uTm9kZTozNDY5',
       'UGVyc29uTm9kZTozNjEx', 'UGVyc29uTm9kZTozMTMy', 'UGVyc29uTm9kZTozNTgx',
       'UGVyc29uTm9kZTozMTUz', 'UGVyc29uTm9kZTozNjk2', 'UGVyc29uTm9kZTozMDc4',
       'UGVyc29uTm9kZTozNjA3'],
      dtype='object', length=396)

In [42]:
#cross check to see that all the observations that have a blank are referred to cases where that specific
#person__id was never associated with a specific political party
for value in aux_CC[:5]:
    aux=original_parliament_data.loc[original_parliament_data.person__id==value].party.value_counts()
    if ("" in aux.index)and(len(aux.index==1)):
        continue
    else:
        print("ERROR")

### 2.2.2 - Creating binary variables for party classification

Filling the maximum number of blanks spaces referred to the party politician association, we created dummy variables for eah party.

In [43]:
parties_1hot=pd.get_dummies(original_parliament_data.party)

original_parliament_data=pd.merge(original_parliament_data.reset_index(),parties_1hot.reset_index(),on="index").set_index("index")

In [44]:
#Cross check to see if the indexes are the same in both dataframes
sum(original_parliament_data.index==parties_1hot.index)/len(original_parliament_data.index==parties_1hot.index)

1.0

### 2.2.3 - Aggregation of parties, focusing on old parties and on left vs. right-wing division

There are other parties being mentioned as the data from newspaper articles starts only in 2007 and the parliament data starts in 1976. Since than many parties party coalitions have been made. Therefore, these extraordinary cases are:

    - ID -Intervenção Democrática (Democratic interventation) -left wing
    - MDPCDE -Movimento Democrático Português/Comissão Democrática Eleitoral (Portuguese Democratic Movement  / Democratic Electoral Comission) - left wing
    - PPM -Partido Popular Monárquico (Monarchic Popular Party) - right wing
    - PRD -Partido Renovador Democrático (Democratic Renewal Party) - right wing
    - PSN -Partido da Solidariedade Nacional (National Solidarity Party) - left wing
    - UDP -União Democrática Popular (Popular Democratic Union) - left wing
    - UEDS -União da Esquerda para a Democracia Socialista (Union of the Left to Socialist Democracy) - left wing
    
    - ADD -no information found
    - DR -no information found
    
    - PPD and PPD/PSD prior PSD
    
    - CDU -coalition between PCP and PEV - left wing
    - FRS -coalition between PS and UEDS - left wing
    - APU -coalition between PCP, PEV and MDPCDE - left wing
    - AD -coalition between PSD, CDS and PPM - right wing
    
In order to aglomerate all the parties and reduce granularity of the data, the older parties that were not overtaken by a current party or coalitions will be divided according to their political orientation: left or right-wing.

The respective left and right-wing classification was based on Wikipedia's classification.

In [45]:
original_parliament_data.loc[:,"left_wing"]=original_parliament_data.loc[:,["ID","MDPCDE","PSN","UDP","UEDS","CDU","FRS","APU"]].sum(axis=1)
original_parliament_data.loc[:,"right_wing"]=original_parliament_data.loc[:,["PPM","PRD","AD"]].sum(axis=1)

In [46]:
original_parliament_data.loc[:,"CDS_final"]=original_parliament_data.loc[:,["CDS","CDS-PP"]].sum(axis=1)
original_parliament_data.loc[:,"PSD_final"]=original_parliament_data.loc[:,["PSD","PPD","PPD/PSD"]].sum(axis=1)
original_parliament_data.loc[:,"other_parties"]=original_parliament_data.loc[:,["ADD","DR"]].sum(axis=1)

In [47]:
original_parliament_data["outros"]=original_parliament_data.agricultura+original_parliament_data.segurança+original_parliament_data.empresas

aux_outros=[]
for value in original_parliament_data.outros:
    if value>=1:
        aux_outros.append(1)
    else:
        aux_outros.append(0)
        
original_parliament_data["outros"]=aux_outros
#this is not made with the parties because they do not face this issue currently

### 2.2.4 - Year, month and trimester classification

In [48]:
original_parliament_data["year"]=pd.DatetimeIndex(original_parliament_data.date).year
original_parliament_data["month"]=pd.DatetimeIndex(original_parliament_data.date).month


In [49]:
original_parliament_data=trimesters(original_parliament_data)

### 2.2.5 - Columns naming adjustments

In [50]:
classified_data_parliament=original_parliament_data.loc[:,['person__id', 'gender', 'party', 'date', 'sustentabilidade', 'JI',
       'ambiente', 'saúde', 'dívida', 'educação', 'SS', 'segurança',
       'empresas', 'agricultura', 'year', 'month','trimester','outros', '',
       'BE', 'PAN', 'PCP','other_parties',
                                                           
       'PEV', 'PS', 'CDS_final', 'PSD_final', 'left_wing','right_wing']].copy()
classified_data_parliament=classified_data_parliament.rename(columns={"CDS_final":"CDS","PSD_final":"PSD"})

In [51]:
rename_topics=dict(zip(['ambiente', 'dívida','educação','saúde','SS', 'outros',],topics_name_w_politics))

classified_data_parliament=classified_data_parliament.rename(columns=rename_topics)
classified_data_parliament=classified_data_parliament.rename(columns={"segurança":"security","empresas":"business","agricultura":"agriculture"})
classified_data_parliament=classified_data_parliament.rename(columns={"sustentabilidade":"SDD"})
classified_data_parliament=classified_data_parliament.rename(columns={"":"no_party_classification"})


### 2.2.6 - New feature: no_class_topic

Several analyses in the remaining documents require the identification of observations that are associated with no party or topic, so these 2 new features were created.

In [52]:
no_class=[]
aux=classified_data_parliament.loc[:,topics_name].sum(axis=1)
for index_,val in enumerate(classified_data_parliament.index):
        if aux.iloc[index_]==0:
            no_class.append(1)
        else:
            no_class.append(0)
            
classified_data_parliament["no_class_topics"]=no_class

### 2.2.7 - Filter for IJ or SDD related observations

Considering that all the analyses focus solely in observations related with:

1. IJ or SDD

The final dataset was filtered considering this restriction.

In [53]:
final_classified_data_parliament=classified_data_parliament[classified_data_parliament.loc[:,["JI","SDD"]].sum(axis=1)!=0]

### 2.2.8 - Download to pickle document

In [54]:
final_classified_data_parliament.to_pickle("parliament_data_SDDJI.pkl")

classified_data_parliament.to_pickle("parliament_data.pkl")

### 2.2.9 - Final Legend

|person_id|gender|party|date|sustentabilidade|JI|environment|health|debt|education|social_security|security|business|agriculture|year|month|trimester|others|no_party_classification|BE|PAN|PCP|other_parties|PEV|PS|CDS|PSD|left_wing|right_wing|no_class_topics|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|Politicians id|Politician's gender (M/F)|Politician's political party|Speech date|Speech classified as SDD related (Y/N)|Speech classified as JI related (Y/N)|Speech classified as environment related (Y/N)|Speech classified as health related (Y/N)|Speech classified as debt related (Y/N)|Speech classified as education related (Y/N)|Speech classified as social security related (Y/N)|Speech classified as security (one of the topic "others") related (Y/N)|Speech classified as companie (one of the topic "others") related (Y/N)|Speech classified as agriculture (one of the topic "others") related (Y/N)|Speech's year|Speech's month|Speech's trimeter|Speech classified as related with topic "others" (Y/N)|Speech not classified with any topic (Y/N)|Speech's politician associated with BE (Y/N)|Speech's politician associated with PAN (Y/N)|Speech's politician associated with PCP (Y/N)|Speech's politician associated with the other parties identified ("ADD" or "DR") (Y/N)|Speech's politician associated with PEV (Y/N)|Speech's politician associated with PS (Y/N)|Speech's politician associated with CDS (Y/N)|Speech's politician associated with PSD (Y/N)|Speech's politician associated with the left wing identified parties ("ID","MDPCDE","PSN","UDP","UEDS","CDU","FRS","APU") (Y/N)|Speech's politician associated with the right wing identified parties ("PPM","PRD","AD") (Y/N)|Speech classified as not related with any topic (Y/N, 1 as not related)|

# 3 - Social Media data (Twitter)

## 3.1 - Data Import

### 3.1.1 - Data upload

The tweets were extracted in 2 parts, hence the existance of 2 datasets.

In [55]:
tweets_data=pd.read_pickle("tweets_data.pkl")

tweets_data_pt=pd.read_pickle("tweets_data_pt.pkl")

### 3.1.2 - Legends

|nbr_retweet|user_id|url|text|usernameTweet|datetime|is_reply|is_retweet|ID|nbr_reply|nbr_favorite|medias|has_media|
|---|---|---|---|---|---|---|---|---|---|---|---|---|
|Number of retweets of observation's tweet|id of the user that published the tweet|Tweet's url text|Tweet's text|Username of the tweet's publisher|Tweet's publication date|Tweet classification as a reply tweet (Y/N)|Tweet classification as a retweet tweet (Y/N)|Tweet's id|Number of reply of the tweet|Number of users that considered the tweet as a favourite tweet|Link mentioned in the tweet|Tweet with an associated link(Y/N)|

## 3.2 - Data Treatment 

### 3.2.1 - Merge datasets

In [56]:
tweets=pd.concat([tweets_data,tweets_data_pt]).reset_index()

Considering the prior merger, and to ensure proper indexes, the index column was created.

In [57]:
tweets=tweets.drop("index",axis=1)
tweets.loc[:,"index"]=range(len(tweets))
tweets=tweets.set_index("index")

### 3.2.2 - Removing duplicates

In [58]:
set(tweets.ID.value_counts().values)

{1}

Since each tweet is associated a single ID, we can ensure that there are no duplicates in the datasets used.

### 3.2.3- Filling missing values and ensure has_media feature coherence

In [59]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69857 entries, 0 to 69856
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   nbr_retweet    69857 non-null  int64 
 1   user_id        69857 non-null  object
 2   url            69857 non-null  object
 3   text           69857 non-null  object
 4   usernameTweet  69857 non-null  object
 5   datetime       69857 non-null  object
 6   is_reply       69857 non-null  bool  
 7   is_retweet     69857 non-null  bool  
 8   ID             69857 non-null  object
 9   nbr_reply      69857 non-null  int64 
 10  nbr_favorite   69857 non-null  int64 
 11  medias         1770 non-null   object
 12  has_media      1770 non-null   object
dtypes: bool(2), int64(3), object(8)
memory usage: 6.5+ MB


In [60]:
tweets.medias.fillna("Nan",inplace=True)

In [61]:
tweets.has_media.fillna(False,inplace=True)

We need to confirm that all columns with True in has_media feature in fact have a link, and vice-versa.

In [62]:
tweets[tweets.has_media==True].medias.value_counts()

Nan                        170
https://t.co/b6MxA0ntNH      7
http://t.co/DqKZLGj4         6
https://t.co/3HbqZ5E9jb      5
https://t.co/YcJuQ2FWop      3
                          ... 
https://t.co/QOTMJkwa1c      1
https://t.co/kdPeLmm8u9      1
http://t.co/mhcSfPwqnd       1
https://t.co/0FQtwGu0bU      1
https://t.co/0iHTBqq2ZQ      1
Name: medias, Length: 1562, dtype: int64

In [63]:
aux_col=[]
for ids,value in enumerate(tweets.index):
    if (tweets.has_media.iloc[ids]==True)&(tweets.medias.iloc[ids]=="Nan"):
        aux_col.append(False)
    else:
        aux_col.append(tweets.has_media.iloc[ids])
        
tweets["has_media"]=aux_col

In [64]:
#cross check
len(tweets[(tweets.has_media==True)&(tweets.medias=="Nan")])

0

In [65]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69857 entries, 0 to 69856
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   nbr_retweet    69857 non-null  int64 
 1   user_id        69857 non-null  object
 2   url            69857 non-null  object
 3   text           69857 non-null  object
 4   usernameTweet  69857 non-null  object
 5   datetime       69857 non-null  object
 6   is_reply       69857 non-null  bool  
 7   is_retweet     69857 non-null  bool  
 8   ID             69857 non-null  object
 9   nbr_reply      69857 non-null  int64 
 10  nbr_favorite   69857 non-null  int64 
 11  medias         69857 non-null  object
 12  has_media      69857 non-null  bool  
dtypes: bool(3), int64(3), object(7)
memory usage: 6.1+ MB


### 3.2.4 - Inserting date columns

In [66]:
tweets["year"]=pd.DatetimeIndex(tweets.datetime).year
tweets["month"]=pd.DatetimeIndex(tweets.datetime).month
tweets=trimesters(tweets)

### 3.2.5 - Converting text to lower case

In [67]:
tweets["text"]=[text.lower() for text in tweets.loc[:,"text"]]


### 3.2.6 - SDD or JI classification

In [68]:
tweets["SDD"]=tweets.text.str.contains("sustentável|insustentável|sustentabilidade|insustentabilidade|sustentar")
tweets["JI"]=tweets.text.str.contains("interger|gerações vindouras|gerações futuras|geração futura|próxima geração|justiça entre gerações|geração vindoura|geração seguinte|gerações presentes|entre gerações|próximas gerações|gerações mais jovens| novas gerações| gerações distintas| próximas gerações") 


### 3.2.7 - SDD or JI related topics classification

In [69]:
def unpack(lists):
    return [i for minilist in lists for i in minilist ] 

In [70]:
business=["organizações","trabalho","empreendedor","investimento"]

society=["cidade","social","popul","transporte","consumidores"]

topics_twitter=unpack([[business,society],topics])

topics_name_w_politics_twitter=unpack([["business","society"],topics_name_w_politics])
topics_name_w_politics_twitter_exp_politics=topics_name_w_politics_twitter[:-1]


In [71]:
tweets_tokens=tweets.text.swifter.apply(word_tokenize)

tweets_topics=tweets.text.swifter.apply(lambda x:values_per_instances(x,topics_twitter,topics_name_w_politics_twitter,1))

topics_count=update_database(tweets_topics,topics_name_w_politics_twitter)

tweets_final=pd.merge(tweets.reset_index(),topics_count.reset_index(),on="index").set_index("index")


Pandas Apply:   0%|          | 0/69857 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/69857 [00:00<?, ?it/s]

In [74]:
booleans_no_class=tweets_final.loc[:,topics_name_w_politics_twitter_exp_politics].sum(axis=1)==0

no_class=[]

for i in booleans_no_class:
    if i:
        no_class.append(1)
    else:
        no_class.append(0)
        
tweets_final.loc[:,"no_class_topic"]=no_class

### 3.2.8 - Filter for IJ or SDD related observations

Considering that all the analyses focus solely in observations related with:

1. IJ or SDD

The final dataset was filtered considering this restriction.

In [75]:
tweets_final_SDDJI=tweets_final[tweets_final.loc[:,["JI","SDD"]].sum(axis=1)!=0]

### 3.2.9 - Download to pickle document

In [76]:
tweets_final.to_pickle("tweets_data_final.pkl")

tweets_final_SDDJI.to_pickle("tweets_data_final_SDDJI.pkl")

### 3.2.10 - Final Legend

|nbr_retweet|user_id|url|text|usernameTweet|datetime|is_reply|is_retweet|ID|nbr_reply|nbr_favorite|medias|has_media|year|month|trimester|SDD|JI|business|society|environment|debt|education|health|social security|others|politics|no_class_topic|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|Number of retweets of observation's tweet|id of the user that published the tweet|Tweet's url text|Tweet's text|Username of the tweet's publisher|Tweet's publication date|Tweet classification as a reply tweet (Y/N)|Tweet classification as a retweet tweet (Y/N)|Tweet's id|Number of reply of the tweet|Number of users that considered the tweet as a favourite tweet|Link mentioned in the tweet|Tweet with an associated link(Y/N)|Tweet's publication year|Tweet's publication month|Tweet's publication trimester|Tweet classified as SDD related (Y/N)|Tweet classified as JI related (Y/N)|Tweet classified as business related (Y/N)|Tweet classified as society related (Y/N)|Tweet classified as environment related (Y/N)|Tweet classified as debt related (Y/N)|Tweet classified as education related (Y/N)|Tweet classified as health related (Y/N)|Tweet classified as social security related (Y/N)|Tweet classified as related with the topic others (Y/N)|Tweet classified as politics related (Y/N)|Tweet classified as not related with any topic (Y/N, 1 as not related)|